# BULK FEATURE EXTRACTION OF THE ML SUBSAMPLE REAL RV CURVES WITH `cesium`

In this notebook we do the bulk feature extraction with `cesium` for all the 233 objects in ML subsample.

**IMPORTANT NOTE:** this code is probably not very efficient (for example, too many dataframe `append` operations, which is costly), but there is no special need at the moment to be more efficient. Maybe the solution is to create a 2D numpy array and then, at the end, create the DataFrame.

## Modules and configuration

### Modules

In [1]:
# Module import:
from IPython.display import clear_output
import warnings
import time

import pandas as pd
import numpy as np

from cesium.data_management import TimeSeries
from cesium.featurize import featurize_single_ts

### Configuration

In [2]:
GTO_FILE = "../data/SELECTION_for_ML_CARM_VIS_objects_with_PG.csv"
#RV_FOLDER = "../data/ML_RVs/"

CS_FEATURES_FOLDER = "../data/DATASETS_CESIUM/"
OUT_DATASET_FILE = "cesium_ML_Subsample_Dataset.csv"

# LIST OF STAR METADATA TO ADD (FROM CARMENCITA DATABASE):
METADATA = ['Karmn', 'SpT', 'SpTnum', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'L_Lsol', 'eL_Lsol',
            'R_Rsol', 'eR_Rsol', 'M_Msol', 'eM_Msol', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'pi_mas',
            'epi_mas', 'd_pc', 'ed_pc', 'Vr_kms-1', 'eVr_kms-1', 'ruwe', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1',
            'W_kms-1', 'eW_kms-1', 'sa_m/s/a', 'esa_m/s/a', 'Pop', 'vsini_flag', 'vsini_kms-1', 'P_d',
            'pEWHalpha_A', 'epEWHalpha_A', 'Activity', 'FUV_mag', 'eFUV_mag', 'NUV_mag', 'eNUV_mag', 'u_mag', 'eu_mag',
            'BT_mag', 'eBT_mag', 'B_mag', 'eB_mag', 'BP_mag', 'eBP_mag', 'g_mag', 'eg_mag', 'VT_mag', 'eVT_mag',
            'V_mag', 'eV_mag', 'Ra_mag', 'r_mag', 'er_mag', 'GG_mag', 'eGG_mag', 'i_mag', 'ei_mag', 'RP_mag', 'eRP_mag',
            'IN_mag', 'J_mag', 'eJ_mag', 'H_mag', 'eH_mag', 'Ks_mag', 'eKs_mag', 'QFlag_2M', 'W1_mag', 'eW1_mag',
            'W2_mag', 'eW2_mag', 'W3_mag', 'eW3_mag', 'W4_mag', 'eW4_mag', 'QFlag_WISE', 'Multiplicity',
            'Planet', 'PlanetNum', 'Teff_min_K', 'Teff_max_K', 'logg_min', 'logg_max', 'is_GTO',
            'InstBand_nominal', 'InstBand_ranged']

# A LIST OF ALL THE FEATURES CESIUM CAN EXTRACT (FOR REFERENCE PURPOSES)
ALL_CS_FEATURES = ['all_times_nhist_numpeaks',
                   'all_times_nhist_peak1_bin', 'all_times_nhist_peak2_bin', 'all_times_nhist_peak3_bin', 'all_times_nhist_peak4_bin',
                   'all_times_nhist_peak_1_to_2', 'all_times_nhist_peak_1_to_3', 'all_times_nhist_peak_1_to_4',
                   'all_times_nhist_peak_2_to_3', 'all_times_nhist_peak_2_to_4',
                   'all_times_nhist_peak_3_to_4',
                   'all_times_nhist_peak_val',
                   'avg_double_to_single_step', 'avg_err', 'avgt',
                   'cad_probs_1', 'cad_probs_10', 'cad_probs_20', 'cad_probs_30', 'cad_probs_40', 'cad_probs_50',
                   'cad_probs_100', 'cad_probs_500', 'cad_probs_1000', 'cad_probs_5000',
                   'cad_probs_10000', 'cad_probs_50000', 'cad_probs_100000', 'cad_probs_500000',
                   'cad_probs_1000000', 'cad_probs_5000000', 'cad_probs_10000000',
                   'cads_avg', 'cads_med', 'cads_std', 'mean',
                   'med_double_to_single_step', 'med_err',
                   'n_epochs', 'std_double_to_single_step', 'std_err',
                   'total_time', 'amplitude',
                   'flux_percentile_ratio_mid20', 'flux_percentile_ratio_mid35', 'flux_percentile_ratio_mid50',
                   'flux_percentile_ratio_mid65', 'flux_percentile_ratio_mid80',
                   'max_slope', 'maximum', 'median', 'median_absolute_deviation', 'minimum',
                   'percent_amplitude', 'percent_beyond_1_std', 'percent_close_to_median', 'percent_difference_flux_percentile',
                   'period_fast', 'qso_log_chi2_qsonu', 'qso_log_chi2nuNULL_chi2nu', 'skew', 'std',
                   'stetson_j', 'stetson_k', 'weighted_average', 'fold2P_slope_10percentile', 'fold2P_slope_90percentile',
                   'freq1_amplitude1', 'freq1_amplitude2', 'freq1_amplitude3', 'freq1_amplitude4',
                   'freq1_freq', 'freq1_lambda', 'freq1_rel_phase2', 'freq1_rel_phase3', 'freq1_rel_phase4', 'freq1_signif',
                   'freq2_amplitude1', 'freq2_amplitude2', 'freq2_amplitude3', 'freq2_amplitude4',
                   'freq2_freq', 'freq2_rel_phase2', 'freq2_rel_phase3', 'freq2_rel_phase4',
                   'freq3_amplitude1', 'freq3_amplitude2', 'freq3_amplitude3', 'freq3_amplitude4',
                   'freq3_freq', 'freq3_rel_phase2', 'freq3_rel_phase3', 'freq3_rel_phase4',
                   'freq_amplitude_ratio_21', 'freq_amplitude_ratio_31',
                   'freq_frequency_ratio_21', 'freq_frequency_ratio_31',
                   'freq_model_max_delta_mags', 'freq_model_min_delta_mags', 'freq_model_phi1_phi2',
                   'freq_n_alias', 'freq_signif_ratio_21', 'freq_signif_ratio_31',
                   'freq_varrat', 'freq_y_offset', 'linear_trend', 'medperc90_2p_p',
                   'p2p_scatter_2praw', 'p2p_scatter_over_mad', 'p2p_scatter_pfold_over_mad', 'p2p_ssqr_diff_over_var',
                   'scatter_res_raw']


## Load ML subsample information table

In [3]:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head()

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS,PG_file_RV,PG_file_TESS
0,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23505-095_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23505-095_RV_PG.dat
1,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23492+024_RV_PG.dat,NaN
2,J23431+365,GJ 1289,-,1289,355.781509,36.536310,23:43:06.31,+36:32:13.1,107.922839,-24.336479,...,999.999512,4.306074e-06,1.0,1.0,NaN,97.939914,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23431+36...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23431+365_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23431+365_RV_PG.dat
3,J23419+441,HH And,-,905,355.480015,44.170376,23:41:55.04,+44:10:38.8,109.989338,-16.947350,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23419+441_RV_PG.dat,NaN
4,J23381-162,G 273-093,-,4352,354.532687,-16.236514,23:38:08.16,-16:14:10.2,61.845437,-69.825220,...,1000.000122,9.022946e-07,1.0,1.0,NaN,136.603404,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23381-16...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23381-162_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23381-162_RV_PG.dat


In [4]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

In [5]:
gto[['rv_file']]

,rv_file
0,../data/CARM_VIS_RVs/J23505-095.avc.dat
1,../data/CARM_VIS_RVs/J23492+024.avc.dat
2,../data/CARM_VIS_RVs/J23431+365.avc.dat
3,../data/CARM_VIS_RVs/J23419+441.avc.dat
4,../data/CARM_VIS_RVs/J23381-162.avc.dat
...,...
228,../data/CARM_VIS_RVs/J00184+440.avc.dat
229,../data/CARM_VIS_RVs/J00183+440.avc.dat
230,../data/CARM_VIS_RVs/J00162+198E.avc.dat
231,../data/CARM_VIS_RVs/J00067-075.avc.dat


## Feature extraction with `cesium` for the real RV curves

In [6]:
# DISABLE WARNINGS:
warnings.filterwarnings('ignore')
# Batch processing:
lapse_list = []
median_lapse = None
# Initialize features dataframe and metafeatures (from disk, or new):
try:
    df = pd.read_csv(CS_FEATURES_FOLDER + OUT_DATASET_FILE, sep=',', decimal='.')
    i0 = len(df)
    print("Previous result found, will continue at record %d..." %len(df))
except:
    # No previous data stored in disk, initialize the DataFrame:
    print("No previous results found, initializing dataframe...")
    df = None
    i0 = 0
metadata_idx = METADATA
#for i in range(i0, 3): # TEST
for i in range(i0, len(gto)):
    clear_output(wait=True)
    start_time = time.time()
    print("Record: %d, started at %s..."
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    # Get metafeatures values:
    metadata_values = list(gto.loc[i, metadata_idx])
    try:
        # load RV file:
        rv = pd.read_csv(gto.loc[i, 'rv_file'], sep=' ', decimal='.',
                         names=['time', 'rv', 'error_rv'])
        # Create TimeSeries object:
        ts = TimeSeries(t=rv['time'], m=rv['rv'], e=rv['error_rv'])
        # Featurize the time series:
        cs = featurize_single_ts(ts, features_to_use=ALL_CS_FEATURES)
        # Join metadata and features for the dataframe:
        indices = metadata_idx + ['VALID_RECORD'] + list(cs.index.get_level_values('feature'))
        values = metadata_values + [True] + list(cs.values)
    except Exception as e:
        # An exception was found, mark the record as invalid and set the features to 'nan':
        print("***ERROR: some error happened in record %d, marking the record as invalid. Error: %s" %(i, str(e)))
        indices = metadata_idx + ['VALID_RECORD'] + ALL_CS_FEATURES
        values = metadata_values + [False] + [np.nan] * 112
    if df is None:
        # Initialize DataFrame (with the first item):
        df = pd.DataFrame(data=[values], columns=indices)
    else:
        # Create a new DataFrame (with the new item):
        new_df = pd.DataFrame(data=[values], columns=indices)
        # Append the new dataframe to the existing one:
        df = df.append(new_df, ignore_index=True)
    # UPDATE THE AVERAGE RECORD PROCESSING TIME:
    lapse = time.time() - start_time
    lapse_list.append(lapse)
    median_lapse = np.nanmedian(lapse_list)
    # Save the results:
    df.to_csv(CS_FEATURES_FOLDER + OUT_DATASET_FILE, sep=',', decimal='.', index=False)


Record: 232, started at 23/01/2023, 13:39:49...
Previous median lapse time: 0.19 seconds


### Next steps are to be executed only if the cell execution is user-interrupted

For example, if the user decided to interrupt the cell execution because it got stuck in some record, the next cells update the info for that record with an "invalid record" mark.

Afterwards, the loop (previous cell) can be executed again and it will start from the record following the problematic one.

## Review the records with errors

In [7]:
df[df['VALID_RECORD'] == False]

,Karmn,SpT,SpTnum,Teff_K,eTeff_K,logg,elogg,[Fe/H],e[Fe/H],L_Lsol,...,freq_signif_ratio_31,freq_varrat,freq_y_offset,linear_trend,medperc90_2p_p,p2p_scatter_2praw,p2p_scatter_over_mad,p2p_scatter_pfold_over_mad,p2p_ssqr_diff_over_var,scatter_res_raw


In [8]:
print(list(df.loc[df['VALID_RECORD'] == False, 'Karmn']))

[]


## Summary

**CONCLUSIONS:**
- Completed the `cesium` feature extraction for all the stars in the ML subsample.
- No one of this objects yielded any error during calculation.